In [3]:
import pandas as pd
import matplotlib as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
Charity_df = pd.read_csv("/Users/luisr/Class/Neural-Networks/Challenge/Resources/charity_data.csv")
Charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
Charity_df.shape


(34299, 12)

In [6]:
Charity_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [7]:
#Removes
Charity_df = Charity_df.drop(["EIN", "NAME","STATUS","SPECIAL_CONSIDERATIONS"], axis = 1)
Charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [8]:
# Generate our categorical variable list
Charity_cat = Charity_df.dtypes[Charity_df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
Charity_df[Charity_cat].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [9]:
# Check the unique value counts to see if binning is required for APPLICATION_TYPE
Application_Type_Count = Charity_df.APPLICATION_TYPE.value_counts()
Application_Type_Count.head(20)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Determine which values to replace for APPLICATION TYPE
Application_Type_Bin =  list(Application_Type_Count[Application_Type_Count < 200].index)
# Replace in DataFrame
for type in Application_Type_Bin:
    Charity_df.APPLICATION_TYPE = Charity_df.APPLICATION_TYPE.replace(type,"Other")

In [11]:
# Check to make sure binning was successful for APPLICATION TYPE
Charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C2500        1
C1236        1
C6100        1
C1728        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# Check the unique value counts to see if binning is required for CLASSIFICATION
Classification_Count = Charity_df.CLASSIFICATION.value_counts()
Classification_Count.head

<bound method NDFrame.head of C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C2500        1
C1236        1
C6100        1
C1728        1
Name: CLASSIFICATION, Length: 71, dtype: int64>

In [13]:
# Determine which values to replace for CLASSIFICATION
Classification_Bin = list(Classification_Count[Classification_Count < 1000].index)
# Replace in DataFrame
for type in Classification_Bin:
    Charity_df.CLASSIFICATION = Charity_df.CLASSIFICATION.replace(type,"Other")

In [14]:
Charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [15]:
# Generate our categorical variable list
Charity_cat = Charity_df.dtypes[Charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
Charity_df[Charity_cat].nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
dtype: int64

In [16]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(Charity_df[Charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(Charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Merge one-hot encoded features and drop the originals
Charity_df = Charity_df.merge(encode_df,left_index=True, right_index=True)
Charity_df = Charity_df.drop(Charity_cat,1)
Charity_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Verify all data in table is numerical
Charity_df.dtypes

ASK_AMT                           int64
IS_SUCCESSFUL                     int64
APPLICATION_TYPE_Other          float64
APPLICATION_TYPE_T10            float64
APPLICATION_TYPE_T19            float64
APPLICATION_TYPE_T3             float64
APPLICATION_TYPE_T4             float64
APPLICATION_TYPE_T5             float64
APPLICATION_TYPE_T6             float64
APPLICATION_TYPE_T7             float64
APPLICATION_TYPE_T8             float64
AFFILIATION_CompanySponsored    float64
AFFILIATION_Family/Parent       float64
AFFILIATION_Independent         float64
AFFILIATION_National            float64
AFFILIATION_Other               float64
AFFILIATION_Regional            float64
CLASSIFICATION_C1000            float64
CLASSIFICATION_C1200            float64
CLASSIFICATION_C2000            float64
CLASSIFICATION_C2100            float64
CLASSIFICATION_C3000            float64
CLASSIFICATION_Other            float64
USE_CASE_CommunityServ          float64
USE_CASE_Heathcare              float64


In [19]:
# Remove IS_SUCCESSFUL target from features data and save as target
y = Charity_df.IS_SUCCESSFUL.values
X = Charity_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# transform the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.468


In [21]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

hidden_nodes =  3 * number_input_features
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes, input_dim=number_input_features, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               4920      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 5,041
Trainable params: 5,041
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  number_input_features*3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
25724/25724 [==============================] - 0s 19us/sample - loss: 0.5715 - acc: 0.7222
Epoch 2/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5577 - acc: 0.7290
Epoch 3/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5535 - acc: 0.7297
Epoch 4/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5523 - acc: 0.7325
Epoch 5/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5506 - acc: 0.7324
Epoch 6/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5500 - acc: 0.7336
Epoch 7/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5499 - acc: 0.7313
Epoch 8/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5486 - acc: 0.7334
Epoch 9/50
25724/25724 [==============================] - 0s 16us/sample - loss: 0.5485 - 

In [23]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = number_input_features*2

nn = tf.keras.models.Sequential()
# 1 hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# 2 hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
25724/25724 [==============================] - 0s 19us/sample - loss: 0.5694 - acc: 0.7231
Epoch 2/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5550 - acc: 0.7311
Epoch 3/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5519 - acc: 0.7312
Epoch 4/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5497 - acc: 0.7325
Epoch 5/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5486 - acc: 0.7330
Epoch 6/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5471 - acc: 0.7345
Epoch 7/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5472 - acc: 0.7347
Epoch 8/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5461 - acc: 0.7356
Epoch 9/50
25724/25724 [==============================] - 0s 17us/sample - loss: 0.5452 - acc: 0.7360
Epoch 10/50
25724/25724 [==============================] - 0s 17us/sample - loss: 